In [1]:
import os 
import datetime
from pathlib import Path
from dotenv import load_dotenv, find_dotenv

basepath = Path(os.getcwd())
# make sure your working directory is the repository root.
if basepath.name != "idp-radio-1":
    os.chdir(basepath.parent.parent.parent)
load_dotenv(find_dotenv())

%load_ext autoreload
%autoreload 2
os.getcwd()

'/srv/idp-radio-1'

In [2]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import classification_report
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras import models
import json
from src.architectures.benchmarks.benchmark import Benchmark, Experiment
from pathlib import Path
from keras.utils.generic_utils import get_custom_objects
from src.metrics.losses import WeightedBinaryCrossentropy, compute_class_weight

Using TensorFlow backend.


In [3]:
# SPECIFY WHICH EXPERIMENT TO LOAD BY INDEX IN EXPERIMENT-LOG.JSON
ind = 25

In [4]:
with open("logs/experiment-log.json") as f:
  experiments = json.load(f)
experiment = experiments['experiments'][ind]
experiment_benchmark = experiments['experiments'][ind]['benchmark']

experiment_benchmark['name'] = experiment_benchmark['benchmark_name']
experiment_benchmark['dataset_folder'] = Path(experiment_benchmark['dataset_folder'])
experiment_benchmark.pop('benchmark_name', None)
experiment_benchmark.pop('train_num_samples', None)
experiment_benchmark.pop('valid_num_samples', None)
experiment_benchmark.pop('test_num_samples', None)

posw = tf.constant(experiment_benchmark['positive_weights'], dtype=tf.float32)
negw = tf.constant(experiment_benchmark['negative_weights'], dtype=tf.float32)

experiment_benchmark.pop('positive_weights', None)
experiment_benchmark.pop('negative_weights', None)

benchmark = Benchmark(**experiment_benchmark)
testgen = benchmark.testgen
traingen = benchmark.traingen
valgen = benchmark.valgen

print(experiment_benchmark['name'])

Chexpert_BCE_E3_B32_C1_N12


In [5]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy

path = os.path.join("models/", experiment['name'], experiment['filename'])
reconstructed_model = models.load_model(path, compile=False)
if experiment_benchmark['loss'] == "weighted_binary_crossentropy":
    loss = WeightedBinaryCrossentropy(posw, negw)
elif experiment_benchmark['loss'] == "binary_crossentropy":
    loss = BinaryCrossentropy()
else:
    raise NotImplementedError()

reconstructed_model.compile(optimizer=Adam(), loss=loss, metrics=[Precision(), Recall()])
path

'models/DenseNet121_Chexpert_BCE_E3_B32_C1_N12/DenseNet121_Chexpert_BCE_E3_B32_C1_N12_20200731-205525.h5'

In [6]:
testgen.on_epoch_end()

In [7]:
thresholds = [0.3, 0.4, 0.5]

In [8]:
compute_class_weight(traingen)

(<tf.Tensor: shape=(12,), dtype=float32, numpy=
 array([20.890194 ,  8.256207 ,  2.1198587, 24.217834 ,  4.2923417,
        15.15646  , 37.428722 ,  6.6715426, 11.323626 ,  2.6077332,
        63.813454 , 25.040297 ], dtype=float32)>,
 <tf.Tensor: shape=(12,), dtype=float32, numpy=
 array([1.050276 , 1.137813 , 1.8929697, 1.0430703, 1.3037351, 1.0706391,
        1.0274509, 1.1763189, 1.0968652, 1.6219938, 1.0159202, 1.0415968],
       dtype=float32)>)

In [9]:
groundtruth_label = testgen.get_labels_nonan()
predictions = reconstructed_model.predict(testgen, steps=len(testgen), verbose=1)

for t in thresholds:
    print('current threshold: ', t)
    predictions_bool = (predictions >= t)
    y_pred = np.array(predictions_bool, dtype=int)

    # report = classification_report(groundtruth_label, y_pred, target_names=list(CHEXPERT_BENCHMARKS['WBCE_E5_B32_C0'].label_columns))
    # print('sklearn report: ', report)
    
    pre = Precision(thresholds=t)
    pre.update_state(groundtruth_label, predictions_bool)
    print('TF Precision: ', pre.result())
    pre.reset_states()
    
    rec = Recall(thresholds=t)
    rec.update_state(groundtruth_label, predictions_bool)
    print('TF Recall: ', rec.result())
    rec.reset_states()
    


44932/44932 [==============================] - 597s 13ms/step
current threshold:  0.3
TF Precision:  tf.Tensor(0.31815258, shape=(), dtype=float32)
TF Recall:  tf.Tensor(0.516681, shape=(), dtype=float32)
current threshold:  0.4
TF Precision:  tf.Tensor(0.32143676, shape=(), dtype=float32)
TF Recall:  tf.Tensor(0.5132978, shape=(), dtype=float32)
current threshold:  0.5
TF Precision:  tf.Tensor(0.32621986, shape=(), dtype=float32)
TF Recall:  tf.Tensor(0.5101889, shape=(), dtype=float32)


In [13]:
predictions
groundtruth_label = testgen.get_labels()

In [14]:
report = classification_report(groundtruth_label, y_pred, target_names=benchmark.label_columns)
print('sklearn report: ', report)

sklearn report:                              precision    recall  f1-score   support

Enlarged Cardiomediastinum       0.00      0.00      0.00      2214
              Cardiomegaly       0.13      0.02      0.04      5294
              Lung Opacity       0.47      1.00      0.64     21324
               Lung Lesion       0.00      0.00      0.00      1901
                     Edema       0.00      0.00      0.00     10461
             Consolidation       0.07      0.84      0.13      3063
                 Pneumonia       0.00      0.00      0.00      1225
               Atelectasis       0.00      0.00      0.00      6912
              Pneumothorax       0.00      0.00      0.00      3894
          Pleural Effusion       0.39      0.80      0.53     17656
             Pleural Other       0.00      0.00      0.00       747
                  Fracture       0.00      0.00      0.00      1863

                 micro avg       0.32      0.50      0.39     76554
                 macro avg   

In [12]:
eval_res = reconstructed_model.evaluate(
            x=testgen, steps=len(testgen), verbose=1)

 2323/44932 [>.............................] - ETA: 10:54 - loss: 4.4079 - precision: 0.3423 - recall: 0.5265

KeyboardInterrupt: 

I think something doesnt work with the masked loss function, I get really low values here. Fix.